# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [ ]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn
import math
import numpy as np
import time
from tqdm.auto import tqdm

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

Источники:
https://towardsdatascience.com/how-to-code-the-transformer-in-pytorch-24db27c8f9ec
https://pytorch.org/tutorials/beginner/transformer_tutorial.html

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) > P(папу | мама, мыла) $$
$$ P(столу | дорога, ложка, к) < P(обеду | дорога, ложка, к) $$
$$ P(Евпатий | меня, зовут) < P(Ваня | меня, зовут) $$
$$ P(журналы | я, часто ,читаю) < P(комиксы | я, часто ,читаю) $$
Попробуйте объяснить выбор для каждого из примеров.

Ответ : 
1) часто цитируемая фраза в народе (имеющие истоки из букваря), стала часто употребляемой.
2) "Дорога ложка к обеду" - поговорка, также часто употребляемая.
3) Имя Ваня более распространенное, чем Евпаторий
4) Лично я чаще читаю комиксы, чем журналы.

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой овтет верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [ ]:
path = '/content/small_corp_for_test.txt'
file = open(path, 'r')
data = file.readlines()
file.close()
len(data)

700000

In [ ]:
data1 =list(map(lambda x: x.replace('\n', ''), data))

## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [ ]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        # YOUR CODE HERE
        text = '['+str.lower(text)+']' + '_'*(window_size - len(text))
        vec1 = [self.token2ind[x] for x in text[:-1]]
        vec2 = [self.token2ind[x] for x in text[1:]]
        return vec2, vec1
        #################

In [ ]:
vec = Preprocessor()

In [ ]:
a, b = vec.preprocess( 'привет',10)

In [ ]:
b

[36, 10, 5, 11, 23, 14, 12, 35, 0, 0, 0]

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

In [ ]:
# YOUR CODE HERE
data2 = ['['+x+']' for x in data1]

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [ ]:
THRESHOLD = 128

# YOUR CODE HERE
data3 = [x for x in data1 if len(x) <= THRESHOLD]
################

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(data3, shuffle = True, random_state = 1, test_size = 0.15)

## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [ ]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 128):
        # YOUR CODE HERE
        self.x = x
        self.p = preproc
        self.w = win_size
        ################
    
    def __len__(self):
        # YOUR CODE HERE
        l = len(self.x)
        return l
        ################
    
    def __getitem__(self, idx):
        x, y = self.p.preprocess(self.x[idx], self.w)
        return torch.LongTensor(x), torch.LongTensor(y)
        ################

In [ ]:
preproc = Preprocessor()
train_dataset = TextDataset(train, preproc)
test_dataset = TextDataset(test, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

## Маскирование матрицы механизма внутреннего внимания позволяет обеспечить то, что элементы будут реагировать только на входные элементы, которые идут в последовательности до них.(https://habr.com/ru/company/wunderfund/blog/593713/) Это необходимо для того, чтобы модель не подглядывала символы, которые будут идти дальше.

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size ,dropout = 0.1):
        super(LanguageModel, self).__init__()
        self.emb = nn.Embedding(vocab_size, 64, padding_idx=0)
        self.pe = PositionalEncoding(64, dropout)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(64,16)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, 6)
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.decoder = nn.Linear(32, vocab_size)
    
    def forward(self, x, src_mask):
        x = self.pe(self.emb(x)) # emb, then pe
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x, src_mask) # transformer encoder with mask
        x = self.decoder(self.relu(self.fc1(x))) # decoder
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
model = LanguageModel(len('_добсркгаупитнезчм яжлйвцыэь-шхющёъ][ '))

In [ ]:
model

LanguageModel(
  (emb): Embedding(38, 64, padding_idx=0)
  (pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
    )
    (linear1): Linear(in_features=64, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=64, bias=True)
    (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = self.train_batch_size)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = self.test_batch_size)
        self.train_dataloader_size = train_dataset.__len__()
        self.test_dataloader_size = test_dataset.__len__()
        
        self.device = 'cuda'
        self.criterion = nn.CrossEntropyLoss() # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                             # ignore index индекс символа _, чтобы модель не штрафовалась за то
                             # что идет после закрывающего токена
        
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        self.steps_to_print = 1000
        
        #self.src_mask = self.model.generate_square_subsequent_mask(128).to(self.device)
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        self.src_mask = self.model.generate_square_subsequent_mask(129).to(self.device)
        for batch in tqdm(self.train_dataloader):
            x, y = batch
            x, y = x.to(self.device), y.to(self.device)
            y_pred = torch.reshape(self.model(x,self.src_mask), (129*y.shape[0],38))
            loss = self.criterion(y_pred,torch.flatten(y)) 
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            step += 1
            counted_loss = loss
            it += 1
            # YOUR CODE HERE
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        self.src_mask = self.model.generate_square_subsequent_mask(129).to(self.device)
        for batch in tqdm(self.test_dataloader):
            x, y = batch
            x, y = x.to(self.device), y.to(self.device)
            y_pred = torch.reshape(self.model(x,self.src_mask), (129*y.shape[0],38))
            loss = self.criterion(y_pred,torch.flatten(y)) 
            step +=1
            counted_loss = loss
            it += 1
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in tqdm(range(1, number_of_epochs+1)):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [22]:
# YOUR CODE HERE
###############
ura_ura = Trainer(model, train_dataset, test_dataset)
ura_ura.train(2)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9085 [00:00<?, ?it/s]

Train epoch 1 | Step 1000/581438 | Counted loss tensor(1.0771, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0010776366302896 | time 281.1432535648346 | 
Train epoch 1 | Step 2000/581438 | Counted loss tensor(0.8743, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0008747102392097 | time 280.12550044059753 | 
Train epoch 1 | Step 3000/581438 | Counted loss tensor(1.0808, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0010814338855711 | time 280.0497815608978 | 
Train epoch 1 | Step 4000/581438 | Counted loss tensor(1.5416, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0015428374767361 | time 279.707302570343 | 
Train epoch 1 | Step 5000/581438 | Counted loss tensor(1.2508, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0012515890821103 | time 279.51680541038513 | 
Train epoch 1 | Step 6000/581438 | Counted loss tensor(1.5333, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0015345058707579 | time 279.60905265808105 | 
Train epoch 1 | Step 7000/581438

  0%|          | 0/1604 [00:00<?, ?it/s]

Validate epoch 1 | Step 500/102607 | Counted loss tensor(12.3305, device='cuda:0') | ppl 1.0249676041729245 | time 47.73219132423401 | 
Validate epoch 1 | Step 1000/102607 | Counted loss tensor(11.9810, device='cuda:0') | ppl 1.0242513807297966 | time 47.661075830459595 | 
Validate epoch 1 | Step 1500/102607 | Counted loss tensor(12.0619, device='cuda:0') | ppl 1.02441722159633 | time 47.680288314819336 | 



  0%|          | 0/9085 [00:00<?, ?it/s]

Train epoch 2 | Step 1000/581438 | Counted loss tensor(1.4930, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0014940977838354 | time 279.9779403209686 | 
Train epoch 2 | Step 2000/581438 | Counted loss tensor(1.2892, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0012900635853967 | time 279.6757116317749 | 
Train epoch 2 | Step 3000/581438 | Counted loss tensor(1.4394, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0014403917993953 | time 279.50328254699707 | 
Train epoch 2 | Step 4000/581438 | Counted loss tensor(1.5107, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0015118231221305 | time 279.6538004875183 | 
Train epoch 2 | Step 5000/581438 | Counted loss tensor(1.2353, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0012361055848238 | time 278.55933952331543 | 
Train epoch 2 | Step 6000/581438 | Counted loss tensor(1.5133, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.001514484205981 | time 280.57573342323303 | 
Train epoch 2 | Step 7000/581438

  0%|          | 0/1604 [00:00<?, ?it/s]

Validate epoch 2 | Step 500/102607 | Counted loss tensor(4.2766, device='cuda:0') | ppl 1.0085899755849967 | time 47.80095362663269 | 
Validate epoch 2 | Step 1000/102607 | Counted loss tensor(4.6674, device='cuda:0') | ppl 1.0093785510273845 | time 47.75260066986084 | 
Validate epoch 2 | Step 1500/102607 | Counted loss tensor(4.6190, device='cuda:0') | ppl 1.0092807052337667 | time 47.74801063537598 | 

